# **Metabolomics Data Visualisation Workflow for ANN-SS (MD-CG) [W/ SMOTE]**

This Google Colab notebook describes the metabolomics data analysis and visualisation workflow for a 2 layer artificial neural network with layer 1 consisting of multiple neurons (n = 2 to 6) with a sigmoidal activation, and layer 2 (output layer) consisting of a single neuron with a sigmoidal activation function (ANN-SS) for a binary classification outcome.

This computational workflow is described using a previously published LC-MS dataset by Sinclair et al. (2021). The study compared the metabolomic profiles across Parkinson's disease patients, characterised as medicated (MD; n=138) and drug-naive (DN; n=80), versus control (CG; n=56) using 8765 named metabolites. For the purpose of this computational workflow, only the MD vs CG samples were compared in a binary discriminant analysis. The deconvolved and annotated data from this study is deposited on Metabolomics Workbench (Study ID: MTBLS2266).

This computational workflow requires a dataset to be in, or converted to, a previously described standardised Excel file format (Mendez et al. 2019). This format uses the Tidy Data Framework (Wickham, 2014), where each row represents an observation (e.g. sample) and each column represents a variable (e.g. age or metabolite). Each excel file (per study) contains two sheets; a data sheet and a peak sheet. The data sheet contains the metabolite concentration together with the metadata associated for each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The peak sheet contains the additional metadata that pertains to the metabolites in the data sheet (requiring the inclusion of the columns: Idx, Name, and Label). The standardisation of this format allows for the efficient re-use of this computational workflow.

The steps included in this data analysis and visualisation workflow are:
1. Import Packages 
2. Load Data and Peak Sheet
3. Data Pre-processing
4. Split Data into Train and Test Set
5. Hyperparameter Optimisation
6. Build Model and Evaluate
7. Permutation Testing
8. Bootstrap Resampling of the Model
9. 

# Section 1 - Import Packages

Certain packages need installing onto the virtual environment prior to use.

For this computational workflow the CIMCB package was installed using pip, CIMCB requires:

*   Python (>=3.5)
*   Bokeh (>=1.0.0)
*   Keras
*   NumPy (>=1.12)
*   SciPy
*   scikit-learn
*   Statsmodels
*   TensorFlow
*   tqdm



In [ ]:
pip install cimcb

To use tools that extend beyond the basic functionalities of Python programming, packages must first be imported to enable their use in each Google Colab environment. Each package is a container of modules.

For this computational workflow, the following packages were used:


*   numpy: A fundamental package for scientific computing with Python, primarly used for the manipulation of arrays
*   pandas: A fundamental package for data analysis and manipulation
*   cimcb: A package for the statistical analysis of untargeted and targeted metabolomics data
*   matplotlib.pyplot: A package mainly used for interactive plots and simple cases of programmatic plot generation
*   seaborn: A package that provides a high-level interface for drawing attractive and informative statistical graphics
*   sklearn: A fundamental package containing tools for machine learning
  *   train_test_split: A method to split arrays into training and test subsets






In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import cimcb as cb

print('All packages successfully loaded')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


All packages successfully loaded


In order to  reproducibility of the workflow, random seeds are set.

*   seed_split: Seed the generator using an integer value e.g. 42 (default = None ; no seed set)

This seed is used to mainatin duplicability in the way the data is divided when the train and test set are generated.

*  seed_init: seed the generator using an integer value e.g. 42 (default = None ; no seed set)

This seed is used to maintain duplicability in the way the intial weights are drawn from a truncated normal distribution when the neural network is first compiled.

In [ ]:
seed_split = 100
seed_init = 4
# seed_split = None
# seed_init = None

# Section 2 - Load Data & Peak Sheet

To upload the dataset to the Google Colab notebook environment, an upload widget was used.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Data and Peak sheet_Test.xlsx to Data and Peak sheet_Test (2).xlsx


The helper function load_dataXL loads the two data sheets from the Excel file 'Data and Peak sheet_Test.xlsx'. Provided the dataset adheres to the standardised TidyData framework format, load_dataXL() outputs the data sheets from the uploaded Excel file as individual Pandas DataFrames.

In [ ]:
# The path to the input file (Excel spreadsheet)
filename = 'Data and Peak sheet_Test.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peaks')

Loadings PeakFile: Peaks
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 418 TOTAL PEAKS: 8765
Done!


# Section 3 - Data Pre-processing

**Section 3.1 - Data Cleaning**

According to Broadhurst (2019), it is best practice to access the quality of the data and refine the dataset by removing those metabolites that lack reporducible measurements. The QC-RSD and percentage of missing values has been calculated and are included in the peakTable DataFrame. Using those values, we remove all metabolomic features that do not meet the following criteria:

*   QC-RSD less than 20%
* Fewer than 10% of values are missing



In [ ]:
# Clean PeakTable
RSD = peakTable['QC_RSD']   
percMiss = peakTable['Perc_missing']  
peakTableClean = peakTable[(RSD < 20) & (percMiss < 10)]   
peakList = peakTableClean['Name']  

print("Number of peaks remaining: {}".format(len(peakTableClean)))

Number of peaks remaining: 2259


**Section 3.2 - Extract X and Y**

As previously mentioned, this workflow is performing binary classification of the classes MD vs CG. The X matrix of metabolite concentrations and Y vector of classification labels ("MD"=0 and "CG"=1) are extracted through the following steps:

1.   Create a subset of the dataTable called dataTable1, containing samples in the Class "MD" or "CG"
2.   Use the peakList variable to hold the names of the metabolites to be used 
3.   Extract all the applicable columns, using peakList, from dataTable1 and place in matrix X
4.   Set Y to the list of binary outcomes from the "Class" column from dataTable1







In [ ]:
# Extract PeakList
dataTable1 = dataTable[(dataTable.Class == "MD") | (dataTable.Class == "CG")]  # Reduce data table only to MD and CG class members
pos_outcome = "CG"

dataTable1['Class'] = [0 if x == 'MD' else 1 for x in dataTable1['Class']]

to_drop = ['Idx', 'SampleID', 'SampleType']
dataTable1.drop(to_drop, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
peaklist = peakTableClean['Name']          
X = dataTable1[peaklist]

merged = pd.concat([dataTable1['Class'], X], axis=1)

merged.reset_index()

,index,Class,M86,M132,M137,M152,M155,M162,M164,M169,...,M8608,M8612,M8642,M8655,M8663,M8685,M8699,M8715,M8729,M8731
0,146,0,31119.367640,641540.06610,26298.038630,5.625178e+06,2.782152e+07,259.651238,483626.43890,73689.639280,...,51430.48636,18412.696190,122545.8321,4.453929e+05,145439.2973,6388.817039,93392.022820,296245.9963,2.652134e+05,39629.25380
1,147,0,27945.701480,106780.30810,19688.706950,8.018405e+05,9.672048e+06,1831.516812,198939.04160,39461.944060,...,35678.47362,10861.174150,239517.2431,8.471789e+05,236214.9011,5990.339636,1753.076374,324820.6171,3.588975e+05,56602.14902
2,148,0,1611.144812,110065.23280,18313.051570,1.243397e+05,2.038063e+06,435001.709000,44944.72675,20217.818250,...,65091.90226,19886.062580,893440.7584,1.808965e+06,295690.8776,87458.523660,6509.821039,725412.7526,4.988300e+05,591755.93280
3,149,0,50276.888560,75422.40729,27554.404720,5.704878e+05,4.083986e+06,358.799341,101763.97820,11822.806800,...,36318.98201,14386.009150,167610.4412,2.782122e+06,223798.5540,8321.018651,1731.627238,228220.1006,3.120556e+05,80658.81383
4,150,0,24708.489720,56223.63935,6637.978372,2.090515e+04,3.006743e+05,146992.465900,61455.45901,8929.524219,...,58034.91423,12801.764260,193332.8822,1.399533e+06,177408.8474,8224.423487,3191.840128,432866.7621,5.516785e+05,83748.00984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,340,1,36243.041440,60373.45415,25849.439280,4.450350e+04,8.392182e+05,145.745213,44730.60527,11237.700140,...,29599.45162,7994.804584,184082.5210,1.808406e+06,176607.4887,8484.157654,3122.291305,371132.4755,3.795142e+05,154225.31570
195,341,1,21490.316750,39058.48983,21279.671080,7.800010e+05,5.580017e+06,90.809228,166831.56020,15793.744980,...,26071.85496,15127.238910,131034.6402,8.734188e+05,257370.1644,6991.514564,1747.801169,162994.3736,2.598393e+05,104217.56760
196,342,1,21108.252930,48508.84804,26621.447530,4.584694e+04,5.648396e+05,326.346111,48897.23332,30104.028980,...,40094.32955,9658.229583,173601.4245,9.854054e+05,314291.2481,12761.289960,2675.477125,207448.2016,5.013632e+05,61234.96429
197,343,1,26654.371260,66551.48742,33271.510880,4.455397e+04,3.834949e+05,1205.570716,33338.19878,43913.138880,...,36357.03474,8987.027989,162060.3538,1.551088e+06,276115.4618,3311.779540,2256.599269,173174.7101,3.910922e+05,77844.74051


**Section 3.3 - Transform and Scale Data**

The MinMaxScaler() method is used to scale all columns that contain values larger than 1, to [0,1] range. This makes the values more manageable when creating and evaluating of the model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Scale only columns that have values greater than 1
to_scale = [col for col in X.columns if X[col].max() > 1]
mms = MinMaxScaler()
scaled = mms.fit_transform(merged[to_scale])
scaled = pd.DataFrame(scaled, columns=to_scale)

# Replace original columns with scaled ones
for col in scaled:
    merged.reset_index()[col] = scaled[col]

In [ ]:
y1 = merged['Class']
merged1 = pd.concat([y1.reset_index()['Class'], scaled], axis=1)

display(merged1)

from sklearn.model_selection import train_test_split

X = merged1.drop('Class', axis=1)
y = merged1['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

print(f'''% Positive class in Train = {np.round(y_train.value_counts(normalize=True)[1] * 100, 2)}
% Positive class in Test  = {np.round(y_test.value_counts(normalize=True)[1] * 100, 2)}''')

,Class,M86,M132,M137,M152,M155,M162,M164,M169,M173,...,M8608,M8612,M8642,M8655,M8663,M8685,M8699,M8715,M8729,M8731
0,0,0.092948,0.001317,0.576761,0.092885,0.097203,0.000450,0.107782,0.003803,0.000772,...,0.069370,0.303060,0.030424,0.000000,0.012907,0.041974,1.000000,0.041004,0.003408,0.029975
1,0,0.083423,0.000182,0.429589,0.013150,0.033477,0.003383,0.043402,0.001793,0.000107,...,0.032643,0.143391,0.117608,0.033392,0.033486,0.037265,0.007049,0.048004,0.018346,0.053870
2,0,0.004386,0.000189,0.398957,0.001950,0.006673,0.811531,0.008577,0.000663,0.000198,...,0.101223,0.334212,0.605004,0.113326,0.046970,1.000000,0.058590,0.146132,0.040657,0.807284
3,0,0.150445,0.000115,0.604737,0.009326,0.013857,0.000635,0.021426,0.000170,0.000194,...,0.034136,0.217920,0.064013,0.194205,0.030671,0.064808,0.006816,0.024341,0.010877,0.087738
4,0,0.073707,0.000074,0.138984,0.000241,0.000573,0.274204,0.012310,0.000000,0.000111,...,0.084769,0.184423,0.083185,0.079298,0.020154,0.063666,0.022639,0.074470,0.049084,0.092087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1,0.108325,0.000083,0.566772,0.000631,0.002464,0.000238,0.008528,0.000136,0.000126,...,0.018469,0.082785,0.076290,0.113280,0.019973,0.066735,0.021885,0.059348,0.021633,0.191309
195,1,0.064049,0.000038,0.465015,0.012789,0.019110,0.000135,0.036141,0.000403,0.000035,...,0.010244,0.233592,0.036751,0.035573,0.038282,0.049096,0.006992,0.008363,0.002551,0.120905
196,1,0.062902,0.000058,0.583963,0.000653,0.001500,0.000575,0.009470,0.001243,0.000003,...,0.042939,0.117956,0.068478,0.044880,0.051187,0.117280,0.017044,0.019253,0.041061,0.060393
197,1,0.079547,0.000096,0.732042,0.000631,0.000864,0.002215,0.005952,0.002054,0.000053,...,0.034225,0.103765,0.059876,0.091894,0.042532,0.005612,0.012505,0.010857,0.023479,0.083777


% Positive class in Train = 29.32
% Positive class in Test  = 25.76


**Section 3.4 - Data Balancing with Missing Values Imputed**

The Synthetic Minority Oversampling Technique (SMOTE) is a statisical method that balances the amount of samples in a dataset by oversampling the minority class. Empty cells are imputed using the .nan_to_num() method.

In [ ]:
from imblearn.over_sampling import SMOTE 

sm = SMOTE(random_state=42)

x = np.nan_to_num(X)
Y = np.nan_to_num(y)

X_sm, y_sm = sm.fit_resample(x, Y)

print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
pd.Series(y_sm).value_counts(normalize=True) * 100

Shape of X before SMOTE: (199, 2259)
Shape of X after SMOTE: (286, 2259)

Balance of positive and negative classes (%):


0    50.0
1    50.0
dtype: float64

# Section 4 - Split Data in Train and Test Set

Using the train_test_split method, the balanced X and Y data is divided into train (2/3) and test (1/3) sets.

In [ ]:
# Optional: Save Class Labels for Figure Legends
Class = merged1.Class

# Split Data into Train (2/3rd) and Test (1/3rd)
XTrain, XTest, YTrain, YTest = train_test_split(X_sm,
                                                y_sm,
                                                test_size=1/3,
                                                random_state=seed_split)

# Section 5 - Hyperparameter Optimisation

**Section 5.1 - k-fold Cross-Validation**

k-fold cross-validation (k=5) is carried out using the CIMCB helper function cb.cross_val.kfold(). This is applied to a set of ANN-SS models with number of neurons ranging from 1 - 6, and a learning rate ranging from 0.01 - 0.05.

In [ ]:
# Parameter Dictionary
lr = [0.01,0.02,0.03,0.04,0.05]
neurons = [2, 3, 4, 5, 6]

param_dict = dict(learning_rate=lr,
                  n_neurons=neurons,
                  epochs=400,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')

# Initialise
cv = cb.cross_val.kfold(model=cb.model.NN_SigmoidSigmoid,
                                X=XTrain,
                                Y=YTrain,
                                param_dict=param_dict,
                                folds=5,
                                n_mc=10)

# Run 
cv.run()

Number of cores set to: 2
Running ...


100%|██████████| 250/250 [50:38<00:00, 12.15s/it]


Time taken: 51.07 minutes with 2 cores
Done!


**Section 5.2 - Plot R^2 and Q^2**

When displaying the R^2 and Q^2 statistics, there are six plots used. From left to right, top to bottom:

1. Heatmap of R^2
2. Heatmap of Q^2
3. Heatmap of 1 - |R^2 - Q^2|
4. |R^2 - Q^2| vs. Q^2
5. R^2 & Q^2 vs. learning rate
6. R^2 & Q^2 vs. number of neurons

The AUC metric portrays the predictability of the model as area under the ROC curve (AUC), AUC(full), and AUC(cv). It is a non-parametric alternative to R^2 and Q^2.

In [ ]:
cv.plot(metric='auc', ci=95)
cv.plot(metric='r2q2', ci=95)

/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()


/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/usr/local/lib/python3.7/dist-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()


**Section 5.3 - Plot Latent Projections: Full and CV**

The method .plot_projections() displays n x n grid of plots, where n is the different number of neurons in the hidden layer being analysed. The types of plot are:

*   Score plots
*   Distribution plots
*   Receiver operating characteristic (ROC) curves

A score plot is produced for each combination of two neurons. Within each score plot the entire score is included, presented as circles, the CV scores, presented as crosses and coloured by group, and the 95% confidence intervals, presented as solid line for full scores and a dashed line for the CV scores. The orthagonal line is displayed as a solid grey line, and the optimal line of seperation as a dashed grey line.

For each neuron a distribution plot is produced. These display the full and CV scores for each group, using kernel density estimation to calculate each distribution.

As with the score plots, the ROC curves produce a plot for each combination of two neurons. The discrimination is determined by the optimal seperation between the two specific neurons being interrogated. Each ROC curve is comprised of a cruve for the full model (green), a curve for the CV model (yellow) with 95% confidence intervals, and the distribution line is presented as a dashed black line.

In [ ]:
cv.plot_projections()

# Section 6 - Build Model and Evaluate

Using the optimal hyperparameter values, identified in Section 5, an ANN-SS model is created and intialised. The model is then trained, where XTrain is the X matrix and YTrain is the Y vector, and tested, where XTest is the X matrix and YTest is y vector, and returns the Y predicted value YPredTest.

The .evaluate() method uses the train and test set to evaluate the predictability of the model. Three plots are produced:

1.   Violin plot
2.   Distribution plot
3.   ROC curve

"The violin plots show the predicted score for the train and test (by group). The distribution plot shows the probability density function of the predicted scores for the train and test (by group). The ROC curve shows the ROC curve for the train (green) and test (yellow)."

In [ ]:
# Build Model
model = cb.model.NN_LogitLogit(learning_rate=0.04,
                                   n_neurons=3,
                                   epochs=400,
                                   momentum=0.5,
                                   decay=0,
                                   loss='binary_crossentropy')
YPredTrain = model.train(XTrain, YTrain)
YPredTest = model.test(XTest)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]


# Evaluate Model (include Test Dataset)
model.evaluate(testset=EvalTest)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


# Section 7 - Permuatation Test

After the model has been trained, permutation testing can be performed to access the reliability of the model. The .permutation_test() method randomises the X matrix, whilst the Y vector remains fixed and is then trained and tested on the randomised data. This is repeated 100 times to produce a more reliable assessment of the distribution of the model.

The method produces two plots:

*   R^2 & Q^2 against the correlation of permuted data against original data
*   Probability densities for R^2 & Q^2

For datasets that contains metabolomic features with no meaningful contribution towards the classification we would expect R^2 & Q^2 values significantly lower than the values from a dataset that contained features with a meaningful contribution.  

In [ ]:
model.permutation_test(nperm=100)

Permutation Resample: 100%|██████████| 100/100 [42:39<00:00, 25.59s/it]


# Section 8 - Bootstrap Resampling of the Model

X

In [ ]:
display(merged1[peakList])

,M86,M132,M137,M152,M155,M162,M164,M169,M173,M176,...,M8608,M8612,M8642,M8655,M8663,M8685,M8699,M8715,M8729,M8731
0,0.092948,0.001317,0.576761,0.092885,0.097203,0.000450,0.107782,0.003803,0.000772,0.003006,...,0.069370,0.303060,0.030424,0.000000,0.012907,0.041974,1.000000,0.041004,0.003408,0.029975
1,0.083423,0.000182,0.429589,0.013150,0.033477,0.003383,0.043402,0.001793,0.000107,0.001022,...,0.032643,0.143391,0.117608,0.033392,0.033486,0.037265,0.007049,0.048004,0.018346,0.053870
2,0.004386,0.000189,0.398957,0.001950,0.006673,0.811531,0.008577,0.000663,0.000198,0.016870,...,0.101223,0.334212,0.605004,0.113326,0.046970,1.000000,0.058590,0.146132,0.040657,0.807284
3,0.150445,0.000115,0.604737,0.009326,0.013857,0.000635,0.021426,0.000170,0.000194,0.009533,...,0.034136,0.217920,0.064013,0.194205,0.030671,0.064808,0.006816,0.024341,0.010877,0.087738
4,0.073707,0.000074,0.138984,0.000241,0.000573,0.274204,0.012310,0.000000,0.000111,0.019813,...,0.084769,0.184423,0.083185,0.079298,0.020154,0.063666,0.022639,0.074470,0.049084,0.092087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.108325,0.000083,0.566772,0.000631,0.002464,0.000238,0.008528,0.000136,0.000126,0.023785,...,0.018469,0.082785,0.076290,0.113280,0.019973,0.066735,0.021885,0.059348,0.021633,0.191309
195,0.064049,0.000038,0.465015,0.012789,0.019110,0.000135,0.036141,0.000403,0.000035,0.006308,...,0.010244,0.233592,0.036751,0.035573,0.038282,0.049096,0.006992,0.008363,0.002551,0.120905
196,0.062902,0.000058,0.583963,0.000653,0.001500,0.000575,0.009470,0.001243,0.000003,0.009306,...,0.042939,0.117956,0.068478,0.044880,0.051187,0.117280,0.017044,0.019253,0.041061,0.060393
197,0.079547,0.000096,0.732042,0.000631,0.000864,0.002215,0.005952,0.002054,0.000053,0.003030,...,0.034225,0.103765,0.059876,0.091894,0.042532,0.005612,0.012505,0.010857,0.023479,0.083777


In [ ]:
display(X_sm)

array([[9.29479391e-02, 1.31744059e-03, 5.76761087e-01, ...,
        4.10042193e-02, 3.40833743e-03, 2.99749870e-02],
       [8.34229174e-02, 1.81668098e-04, 4.29588878e-01, ...,
        4.80037865e-02, 1.83458471e-02, 5.38702126e-02],
       [4.38586325e-03, 1.88644927e-04, 3.98956698e-01, ...,
        1.46131828e-01, 4.06574921e-02, 8.07284418e-01],
       ...,
       [6.63217551e-02, 3.97182956e-04, 8.61602638e-01, ...,
        1.79933884e-02, 6.59172739e-02, 5.46046271e-02],
       [2.94018489e-02, 9.17960266e-04, 6.82602894e-01, ...,
        8.81390281e-02, 5.26202959e-02, 9.55225010e-02],
       [6.41448355e-02, 6.98888726e-05, 5.85803777e-01, ...,
        2.03731659e-02, 4.01541892e-02, 6.45016518e-02]])

In [ ]:
from keras.utils.generic_utils import default

YPredBoot = model.train(X_sm, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.BCA(model, XBootKnn, Y, bootlist=YPredBoot, bootnum=100)
bootmodel.run()

/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1545: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')
/usr/local/lib/python3.7/dist-packages/cimcb/utils/scale.py:69: RuntimeWarning: invalid value encountered in subtract
  z = (x - mu) / sigma


ValueError: ignored